In [2]:
import os, glob, random, cv2
import keras
import multiprocessing

import numpy as np
import pandas as pd
import keras.backend as K
import matplotlib.pyplot as plt

from copy import deepcopy

from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split

from keras.optimizers import Adam
from keras.callbacks import Callback
from keras.applications.densenet import DenseNet201
from keras.layers import Dense, Flatten
from keras.models import Model, load_model
from keras.utils import Sequence
from keras_radam import RAdam

from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
from IPython.display import Image
from tqdm import tqdm_notebook as tqdm
from numpy.random import seed
import tensorflow

seed(10)
tensorflow.random.set_seed(10)
%matplotlib inline

In [3]:
test_imgs_dir = 'C:/Users/priya/Downloads/understanding_cloud_organization/test_images'
t_image_dir ='C:/Users/priya/Downloads/understanding_cloud_organization/train_images'
num_cores = multiprocessing.cpu_count()

In [4]:
train_df = pd.read_csv('C:/Users/priya/Downloads/understanding_cloud_organization/train.csv')
train_df.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


In [5]:
t_data_F = t_data_F[~t_data_F['EncodedPixels'].isnull()]
t_data_F['Image'] = t_data_F['Image_Label'].map(lambda x: x.split('_')[0])
t_data_F['Class'] = t_data_F['Image_Label'].map(lambda x: x.split('_')[1])

cls = t_data_F['Class'].unique()

t_data_F = t_data_F.groupby('Image')['Class'].agg(set).reset_index()

for class_name in cls:
    t_data_F[class_name] = t_data_F['Class'].map(lambda x: 1 if class_name in x else 0)

t_data_F.head()

,Image,Class,Fish,Flower,Sugar,Gravel
0,0011165.jpg,"{Fish, Flower}",1,1,0,0
1,002be4f.jpg,"{Fish, Flower, Sugar}",1,1,1,0
2,0031ae9.jpg,"{Fish, Flower, Sugar}",1,1,1,0
3,0035239.jpg,"{Flower, Gravel}",0,1,0,1
4,003994e.jpg,"{Fish, Gravel, Sugar}",1,0,1,1


In [6]:
# Dictionary for fast access
img_2_ohe_vector = {img:vec for img, vec in zip(train_df['Image'], train_df.iloc[:, 2:].values)}

In [7]:
# train-val split
t_image, validation_image = train_test_split(train_df['Image'].values, 
                                        test_size = 0.2, 
                                        stratify = train_df['Class'].map(lambda x: str(sorted(list(x)))), 
                                        random_state = 2019)

In [8]:
class DG(Sequence):
    def __init__(self, images_list = None, folder_imgs = t_image_dir, 
                 batch_size = 32, shuffle = True, augmentation = None,
                 resized_height = 260, resized_width = 260, num_channels = 3):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        
        if images_list is None:
            self.images_list = os.listdir(folder_imgs)
        else:
            self.images_list = deepcopy(images_list)
        
        
        self.folder_imgs = folder_imgs
        self.len = len(self.images_list) // self.batch_size
        self.resized_height = resized_height
        self.resized_width = resized_width
        self.num_channels = num_channels
        self.num_classes = 4
        self.is_test = not 'train' in folder_imgs
        
        if not shuffle and not self.is_test:
            self.labels = [img_2_ohe_vector[img] for img in self.images_list[:self.len * self.batch_size]]

    def __len__(self):
        return self.len
    
    def epoch_start(self):
        if self.shuffle:
            random.shuffle(self.images_list)

    def __getitem__(self, idx):
        current_batch = self.images_list[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, self.resized_height, self.resized_width, self.num_channels))
        y = np.empty((self.batch_size, self.num_classes))

        for i, image_name in enumerate(current_batch):
            path = os.path.join(self.folder_imgs, image_name)
            img = cv2.resize(cv2.imread(path), (self.resized_height, self.resized_width)).astype(np.float32)
            
            if not self.augmentation is None:
                augmented = self.augmentation(image=img)
                img = augmented['image']
            
            X[i, :, :, :] = img/255.0
            
            if not self.is_test:
                y[i, :] = img_2_ohe_vector[image_name]
        
        return X, y

    # get label
    def g_label(self):
        if self.shuffle:
            images_current = self.images_list[:self.len * self.batch_size]
            labels = [img_2_ohe_vector[img] for img in images_current]
        else:
            labels = self.labels
        return np.array(labels)

In [9]:
albumentations_train = Compose([ VerticalFlip(), HorizontalFlip(), Rotate(limit=20), GridDistortion() ], p = 1)

In [10]:
# generator instances
data_generator_train = DG(t_image, augmentation = albumentations_train)
data_generator_train_eval = DG(t_image, shuffle = False)
data_generator_val = DG(validation_image, shuffle = False)

In [11]:
class PR(Callback):
    def __init__(self, data_generator, num_workers = num_cores, 
                 early_stopping_pt = 5, 
                 plateau_pt = 3, reduction_rate = 0.5,
                 stage = 'train', checkpoints_path = 'checkpoints/'):
        super(Callback, self).__init__()
        self.data_generator = data_generator
        self.num_workers = num_workers
        self.class_names = ['Fish', 'Flower', 'Sugar', 'Gravel']
        self.history = [[] for _ in range(len(self.class_names) + 1)] # to store per each class and also mean PR AUC
        self.early_stopping_pt = early_stopping_pt
        self.plateau_pt = plateau_pt
        self.reduction_rate = reduction_rate
        self.stage = stage
        self.best_pr_auc = -float('inf')
        
        if not os.path.exists(checkpoints_path):
            os.makedirs(checkpoints_path)
        
        self.checkpoints_path = checkpoints_path
        
    def c_auc(self, y_true, y_pred):
        pr_auc_mean = 0
        for class_i in range(len(self.class_names)):
            precision, recall, _ = precision_recall_curve(y_true[:, class_i], y_pred[:, class_i])
            pr_auc = auc(recall, precision)
            pr_auc_mean += pr_auc/len(self.class_names)
            print(f"PR-AUC {self.class_names[class_i]}, {self.stage}: {pr_auc:.3f}")
            self.history[class_i].append(pr_auc)        
        print(f"\nmean, {self.stage}: {pr_auc_mean:.3f}")
        self.history[-1].append(pr_auc_mean)
        return pr_auc_mean
              
    def check_pt(self, pt):
        if len(self.history[-1]) > pt:
            best_performance = max(self.history[-1][-(pt + 1):-1])
            return best_performance == self.history[-1][-(pt + 1)] and best_performance >= self.history[-1][-1]    
              
    def che(self, pr_auc_mean):
        if self.check_pt(self.early_stopping_pt):
            self.model.stop_training = True    
              
    def model_checkpoint(self, pr_auc_mean, epoch):
        if pr_auc_mean > self.best_pr_auc:
            
            # remove previous checkpoints to save space
            for checkpoint in glob.glob(os.path.join(self.checkpoints_path, 'classifier_epoch_*')):
                os.remove(checkpoint)
            self.best_pr_auc = pr_auc_mean
            self.model.save(os.path.join(self.checkpoints_path, f'classifier_epoch_{epoch}_val_pr_auc_{pr_auc_mean}.h5'))              
            print(f"\nSaved new checkpoint")
              
    def reduce_lr_on_plateau(self):
        if self.check_pt(self.plateau_pt):
            new_lr = float(keras.backend.get_value(self.model.optimizer.lr)) * self.reduction_rate
            keras.backend.set_value(self.model.optimizer.lr, new_lr)
            print(f"\nReduced learning rate to {new_lr}.")
        
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)
        y_true = self.data_generator.g_label()
        
        # estimate AUC under precision recall curve for each class
        pr_auc_mean = self.c_auc(y_true, y_pred)
              
        if self.stage == 'val':
            self.che(pr_auc_mean)
            self.model_checkpoint(pr_auc_mean, epoch)
            self.reduce_lr_on_plateau()            
        
    def get_pr_auc_history(self):
        return self.history

In [12]:
train_metric_callback = PR(data_generator_train_eval)
val_callback = PR(data_generator_val, stage='val')

In [13]:
# CLASSIFIER
# dice loss
from keras.losses import binary_crossentropy
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def d_c(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_d_c(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + d_c(y_true, y_pred)

In [15]:
import efficientnet.keras as efn 
def get_model():
    K.clear_session()
    base_model =  efn.EfficientNetB0(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB2(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB3(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB4(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
#     base_model =  efn.EfficientNetB5(weights='imagenet', include_top=False, pooling='avg', input_shape=(260, 260, 3))
    x = base_model.output
    y_pred = Dense(4, activation='sigmoid')(x)
    return Model(inputs = base_model.input, outputs = y_pred)

model = get_model()

16804768/16804768 [==============================] - 10s 1us/step


In [16]:
# Initial tuning of the added fully-connected layer
for base_layer in model.layers[:-3]:
    base_layer.trainable = False
    
model.compile(optimizer = RAdam(warmup_proportion = 0.1, min_lr = 1e-5),  
                              loss = 'categorical_crossentropy', 
                              metrics = ['accuracy'])

history_0 = model.fit_generator(generator = data_generator_train,
                              validation_data = data_generator_val,
                              epochs = 20,
                              callbacks = [train_metric_callback, val_callback],
                              workers = num_cores,
                              verbose = 1)

C:\Users\priya\AppData\Local\Temp/ipykernel_28504/2165960536.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_0 = model.fit_generator(generator = data_generator_train,


Epoch 1/20
138/138 [==============================] - ETA: 0s - loss: 3.0742 - accuracy: 0.1377

C:\Users\priya\AppData\Local\Temp/ipykernel_28504/735188133.py:60: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)


PR-AUC Fish, train: 0.518
PR-AUC Flower, train: 0.555
PR-AUC Sugar, train: 0.668
PR-AUC Gravel, train: 0.585

mean, train: 0.581


C:\Users\priya\AppData\Local\Temp/ipykernel_28504/735188133.py:60: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)


...

PR-AUC Fish, val: 0.752
PR-AUC Flower, val: 0.713
PR-AUC Sugar, val: 0.839
PR-AUC Gravel, val: 0.764

mean, val: 0.767

Reduced learning rate to 0.0002500000118743628.
138/138 [==============================] - 368s 3s/step - loss: 2.8365 - accuracy: 0.3102 - val_loss: 2.8304 - val_accuracy: 0.3676
Epoch 8/20
138/138 [==============================] - ETA: 0s - loss: 2.8310 - accuracy: 0.3170

C:\Users\priya\AppData\Local\Temp/ipykernel_28504/735188133.py:60: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)


PR-AUC Fish, train: 0.751
PR-AUC Flower, train: 0.713
PR-AUC Sugar, train: 0.838
PR-AUC Gravel, train: 0.765

mean, train: 0.767


C:\Users\priya\AppData\Local\Temp/ipykernel_28504/735188133.py:60: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)


PR-AUC Fish, val: 0.752
PR-AUC Flower, val: 0.713
PR-AUC Sugar, val: 0.839
PR-AUC Gravel, val: 0.764

mean, val: 0.767

Reduced learning rate to 0.0001250000059371814.
138/138 [==============================] - 367s 3s/step - loss: 2.8310 - accuracy: 0.3170 - val_loss: 2.8292 - val_accuracy: 0.3594


In [17]:
for base_layer in model.layers[:-3]:
    base_layer.trainable = True
    
model.compile(optimizer = RAdam(warmup_proportion = 0.1, min_lr = 1e-5),  
                              loss = 'categorical_crossentropy', 
                              metrics = ['accuracy'])
history_1 = model.fit_generator(generator = data_generator_train,
                              validation_data = data_generator_val,
                              epochs = 20,
                              callbacks = [train_metric_callback, val_callback],
                              workers = num_cores,
                              verbose = 1,
                              initial_epoch = 1)

C:\Users\priya\AppData\Local\Temp/ipykernel_28504/1519345561.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_1 = model.fit_generator(generator = data_generator_train,


Epoch 2/20
138/138 [==============================] - ETA: 0s - loss: 3.4633 - accuracy: 0.2810

C:\Users\priya\AppData\Local\Temp/ipykernel_28504/735188133.py:60: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)


PR-AUC Fish, train: 0.751
PR-AUC Flower, train: 0.713
PR-AUC Sugar, train: 0.838
PR-AUC Gravel, train: 0.765

mean, train: 0.767


C:\Users\priya\AppData\Local\Temp/ipykernel_28504/735188133.py:60: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = self.model.predict_generator(self.data_generator, workers=self.num_workers)


PR-AUC Fish, val: 0.752
PR-AUC Flower, val: 0.713
PR-AUC Sugar, val: 0.839
PR-AUC Gravel, val: 0.764

mean, val: 0.767

Reduced learning rate to 0.0005000000237487257.
138/138 [==============================] - 1042s 7s/step - loss: 3.4633 - accuracy: 0.2810 - val_loss: 2.9733 - val_accuracy: 0.2059
